In [1]:
import pandas as pd
from datetime import datetime


<h4>Read FIN001 sample report and paycheck table</h4>

In [2]:
#FIN001filepath = r"C:\Users\awidjaja\Documents\RFL- POC\Input_Files\UC_FIN001_PAYCHECK_DEDUCTIONS_774084231.xls"
FIN001filepath =r"C:\Users\awidjaja\Documents\RFL- POC\FIN001\FIN001_1.xls"
df = pd.read_excel(FIN001filepath, header=1)
paycheckfilepath=r"C:\Users\awidjaja\Documents\RFL- POC\Input_Files\BiWeekly Paycheck Dates.xlsx"
dfPaycheck = pd.read_excel(paycheckfilepath)

<h4>Identifying Variances per Year per Plan Type </h4>

In [ ]:
selectedPlanTypes=[
                "Legal Insurance",
                "Supplemental Life",
                "Employee & Dependent AD&D",
                "Basic Dependent Life",
                "Exp Dependent Life - Spouse/DP",
                "Exp Dependent Life - Child",
                "Voluntary Long-Term Disability",
                "Voluntary Short-Term Disability",
                "Accident",
                "Critical Illness EE",
                "Critical Illness SP",
                "Hospital Indemnity",
                "Dental",
                "Vision",
                "Life",
                "Basic Disability"
]
df['Paycheck Year'] = df['Paycheck Issue Date'].dt.year
df['Paycheck Month'] = df['Paycheck Issue Date'].dt.month
df = df.sort_values(by=['Plan Type', 'Paycheck Year','Paycheck Issue Date'])
df['Deduction Variance'] = (df.groupby(['Paycheck Year','Plan Type'])['Current Deduction'].diff())

#### THIS ONLY APPLIES TO BIWEEKLY
df['Deduction Variance'] = df.apply(
    lambda row: None if row['Paycheck Month'] == 12 else row['Deduction Variance'],
    axis=1
)
### ------------ ####
#### Monthly ###
"""
df['Deduction Variance'] = df.apply(
    lambda row: None if ~(row['Plan Type'].isin(selectedPlanTypes)) else row['Deduction Variance'],
    axis=1
)
"""
#####
t_year ="2020"
f_year = "%Y"
d_year = datetime.strptime(t_year,f_year)
print(d_year)
df[['Paycheck Issue Date','Plan Type','Current Deduction','Deduction Variance','Paycheck Year']][(df['Paycheck Year']==d_year.year)]

In [14]:

def highlightVariance(row):
    if (
        pd.notna(row['Deduction Variance']) and 
        row['Deduction Variance'] != 0.0 and
        row['Plan Type'] in selectedPlanTypes
    ):
        return['background-color:pink' if col == 'Current Deduction' else '' for col in row.index]
    else:
        return ['' for _ in row.index]
df= df.sort_index()
dfstyler222= df.style.apply(highlightVariance, axis=1)


In [6]:
dfstyler222.to_html("FIN001_01_Variance_Detection_selectedPlanTypes.html")

In [7]:
dfstyler222.to_excel("./test_color.xlsx",index=False)

<h4>Identifying Missing Dates</h4>

<h5>Finding out the existing date before the missed date -- Not Necessary </h5>

In [ ]:
#pdBefore
#once we get the dates of paycheck received before missed dates, we need to substract the known missing dates from the before dates

indexBefore =[]
pdBefore =[]
counter =0
for pc in dfBiWeeklyPaycheckDates:
    for smd in missingDates:
        if smd == pc:
            indexNumberBefore = counter -1
            indexBefore.append(indexNumberBefore)
            pdBefore.append(dfBiWeeklyPaycheckDates[indexNumberBefore])
    counter += 1
        
        

truePaycheckDateBefore = sorted(list(set(pdBefore) - set(missingDates)))
for bdt in truePaycheckDateBefore:
    t_bdt = datetime.strftime(bdt,"%Y-%m-%d")
    print(t_bdt)

<h5>Inserting missing dates for Plan Types not Dental, Vision, Basic Disability, and create Dataframe</h5>

In [25]:
def insertMissingDates(group,Calendardf):
    Calendar = pd.Series(Calendardf[(Calendardf >=group['Paycheck Issue Date'].min())&(Calendardf<=group['Paycheck Issue Date'].max())])
    
    missingDates = sorted(list(set(Calendar)-set(group['Paycheck Issue Date'])))
    newRows =[]
    #insert the missing dates:
    if len(missingDates)>0:
        for missedDate in missingDates:
            #print(group[['Business Unit','Plan Type']].iloc[0])
            #print(group['Business Unit'].loc[0])
            #======================================================
            # added 10/27/2025 at 8:45 am
            missedDateMonth = missedDate.month
            missedDateYear = missedDate.year
            #check the year of the known value where paycheck issue date year in front and back
            frontKnownExistedRowYear = group['Paycheck Issue Date'][(group['Paycheck Issue Date'])< missedDate].iloc[-1].year

            #if December (or 12), see the row after i.e. move forward
            if (missedDateMonth == 12):
                 # Get the known date after the missed date. the known value is at the top of the array, so iloc[0] should get the value on top of array  
                existedRowDate = group['Paycheck Issue Date'][(group['Paycheck Issue Date'] >missedDate)].iloc[0] 
            elif(missedDateYear == frontKnownExistedRowYear):
                # Get the known date after the missed date. the known value is at the bottom of the array, so iloc[-1] should get the value on bottom of array  
                existedRowDate = group['Paycheck Issue Date'][(group['Paycheck Issue Date'] <missedDate)].iloc[-1]
            else:
                # Get the known date after the missed date. the known value is at the top of the array, so iloc[0] should get the value on top of array  
                existedRowDate = group['Paycheck Issue Date'][(group['Paycheck Issue Date'] >missedDate)].iloc[0] 
            
            prior_row = group[(group["Paycheck Issue Date"] == existedRowDate)]
            #print(prior_row)
            if not prior_row.empty:
                if len(prior_row)>1:
                    for indexNum in range(len(prior_row)):
                        pr = prior_row.iloc[indexNum]
                        new_row ={
                            'Business Unit':pr['Business Unit'],
                            'Employee ID':pr['Employee ID'],
                            'Employee Record':pr['Employee Record'],
                            'Employee Name':pr['Employee Name'],
                            'Pay Group':pr['Pay Group'],
                            'Pay Period End Date':pr['Pay Period End Date'],
                            'Paycheck Issue Date':missedDate,
                            'Employee Type':pr['Employee Type'],
                            'Plan Type':pr['Plan Type'],
                            'Benefit Plan':pr['Benefit Plan'],
                            'Deduction Code':pr['Deduction Code'],
                            'Deduction Code Descr':pr['Deduction Code Descr'],
                            'Deduction Classification':pr['Deduction Classification'],
                            'Current Deduction':pr['Current Deduction'],
                            'Payback Amount':pr['Payback Amount'],
                            'Refund Amount':pr['Refund Amount'],
                            'Not Taken':pr['Not Taken'],
                            'Reason':pr['Reason'],
                            'Calculated Base':pr['Calculated Base'],
                            'Paycheck Status':pr['Paycheck Status'],
                            'Paycheck Option':pr['Paycheck Option'],
                            'Off Cycle':pr['Off Cycle'],
                            'Is New Row':True
                        }
                        newRows.append(new_row)
                else:
                    prIloc0 = prior_row.iloc[0]
                    new_row2 ={
                        'Business Unit':prIloc0['Business Unit'],
                        'Employee ID':prIloc0['Employee ID'],
                        'Employee Record':prIloc0['Employee Record'],
                        'Employee Name':prIloc0['Employee Name'],
                        'Pay Group':prIloc0['Pay Group'],
                        'Pay Period End Date':prIloc0['Pay Period End Date'],
                        'Paycheck Issue Date':missedDate,
                        'Employee Type':prIloc0['Employee Type'],
                        'Plan Type':prIloc0['Plan Type'],
                        'Benefit Plan':prIloc0['Benefit Plan'],
                        'Deduction Code':prIloc0['Deduction Code'],
                        'Deduction Code Descr':prIloc0['Deduction Code Descr'],
                        'Deduction Classification':prIloc0['Deduction Classification'],
                        'Current Deduction':prIloc0['Current Deduction'],
                        'Payback Amount':prIloc0['Payback Amount'],
                        'Refund Amount':prIloc0['Refund Amount'],
                        'Not Taken':prIloc0['Not Taken'],
                        'Reason':prIloc0['Reason'],
                        'Calculated Base':prIloc0['Calculated Base'],
                        'Paycheck Status':prIloc0['Paycheck Status'],
                        'Paycheck Option':prIloc0['Paycheck Option'],
                        'Off Cycle':prIloc0['Off Cycle'],
                        'Is New Row':True
                    }
                    newRows.append(new_row2)
    return newRows

In [17]:
"""
selectedPlans =[
"Accident",
"Basic Dependent Life",
"Critical Illness - EE (+Ch)",
"Critical Illness - SP/DP",
"Employee & Dependent AD&D",
"Exp Dependent Life - Child",
"Exp Dependent Life - Spouse/DP",
"General Deduction",
"Hospital Indemnity",
"Legal Insurance",
"Medical",
"Supplemental Life",
"Voluntary LongTerm Disability",
"Voluntary ShortTerm Disability"
]
"""
selectedPlans =['Medical','Legal Insurance','Critical Illness - EE (+Ch)','General Deduction','Dental','Vision']
# create 2 dataframes: Selected vs Other
dfSelected = df[(df['Plan Type'].isin(selectedPlans))]
dfOther =df[~(df['Plan Type'].isin(selectedPlans))]


for eachPlan in dfSelected['Plan Type'].unique():
    if eachPlan in(['Dental','Life','Basic Disability','Vision']):
        lInsertedMissingDates= insertMissingDates(dfSelected[(dfSelected['Plan Type']==eachPlan)],dfPaycheck['Bi-Weekly Paycheck Dates for Dental/Vision/Basic Disability'])
    else:
        lInsertedMissingDates= insertMissingDates(dfSelected[(dfSelected['Plan Type']==eachPlan)],dfPaycheck['Bi-Weekly Paycheck Dates'])
    newPD = pd.DataFrame(lInsertedMissingDates)
    dfSelected= pd.concat([dfSelected,newPD], ignore_index=True)


""


In [42]:
planVDBLBD = ['Vision','Dental','Life','Basic Disability']
allPaycheckDatesforVDBLBD = pd.to_datetime(df['Paycheck Issue Date'][(df['Plan Type'].isin(planVDBLBD))].unique())
sortedallPaycheckDatesforVDBLBD = sorted(allPaycheckDatesforVDBLBD)
missingDates = sorted(list(set(dfBiWeeklyPDDVB)-set(sortedallPaycheckDatesforVDBLBD)))

listPDVDBLBD = createListMissedPaychecks(df[(df['Plan Type'].isin(planVDBLBD))],missingDates,planVDBLBD) 

In [43]:
missingDates

[Timestamp('2020-03-04 00:00:00'),
 Timestamp('2020-04-01 00:00:00'),
 Timestamp('2020-05-13 00:00:00'),
 Timestamp('2022-07-06 00:00:00')]

<h4>Highlight variances on Payback Amount, Refund Amount, Not Taken, Reason , Paycheck Status , Paycheck Option, OffCycle</h4>
<p>Conditions are as follow:</p>
<ul>
    <li>if Payback Amount (> or <) 0.00 then color cell </li>
    <li>if Refund Amount (> or <) 0.00, then color cell </li>
    <li>if Not Taken (> or <) 0.00, then color cell </li>
    <li>if Reason != null, then color cell </li>
    <li>if Paycheck Status != 'Confirmed', then color cell </li>
    <li>if Paycheck Options != 'Advice', then color cell </li>
    <li>if Off Cycle != 'N'</li>
</ul>

In [ ]:
df['Payback Amount'][0] = 3
df['Refund Amount'][10] = 33.5
df['Not Taken'][11] = 270
df['Not Taken'][12] = -250
df['Reason'][12] = "Some random reasons..."
df['Paycheck Status'][13] = "NOT CONFIRMED"
df['Paycheck Option'][13] = "Not Advice"
df['Off Cycle'][20] ='Y'

In [16]:
df.fillna("Null",inplace=True)

<p>Note about Styler object:</p>
<ol>
<li>When you call df.style, it returns a pandas.io.formats.style.Styler object, not a DataFrame.</li>
<li>The Styler object provides methods for applying various styles (e.g., highlighting, formatting, coloring) to the DataFrame's display.</li>
<li>It is not intended for data selection or modification, which are operations performed on the underlying DataFrame itself.</li>
</ol>
    <p>Solution: To resolve this error, ensure that you are performing data selection or manipulation on the original DataFrame before applying styling, or access the underlying DataFrame data from the Styler object if necessary.</p>
    <p>In summary, the Styler object is for presentation and not for data manipulation using subscripting. Always ensure you are interacting with the appropriate object (DataFrame for data, Styler for styling) for the intended operation.</p>


In [17]:
def highlightAnomalies(val, col_name):
    if col_name == 'Payback Amount' and (val >0 or val <0):
        return 'background-color: pink'
    elif col_name =='Refund Amount' and (val > 0 or val<0):
        return 'background-color:pink'
    elif col_name =='Not Taken' and (val > 0 or val<0):
        return 'background-color:pink'
    elif col_name == 'Reason' and val != "Null":
        return 'background-color:pink'
    elif col_name =='Paycheck Status' and val !="Confirmed":
        return 'background-color:pink'
    elif col_name =='Paycheck Option' and val !="Advice":
        return 'background-color:pink'
    elif col_name =='Off Cycle' and val !="N":
        return 'background-color:pink'
    else:
        return ''
    

dfStyler = df.style.applymap(lambda x: highlightAnomalies(x, 'Payback Amount'), subset=['Payback Amount'])\
.applymap(lambda x: highlightAnomalies(x, 'Refund Amount'), subset=['Refund Amount'])\
.applymap(lambda x: highlightAnomalies(x, 'Not Taken'), subset=['Not Taken'])\
.applymap(lambda x: highlightAnomalies(x, 'Reason'), subset=['Reason'])\
.applymap(lambda x: highlightAnomalies(x, 'Paycheck Status'), subset=['Paycheck Status'])\
.applymap(lambda x: highlightAnomalies(x, 'Paycheck Option'), subset=['Paycheck Option'])\
.applymap(lambda x: highlightAnomalies(x, 'Off Cycle'), subset=['Off Cycle'])




In [ ]:
dfStyler